# Strands Agents におけるツールとしてのエージェント

「ツールとしてのエージェント（Agents as Tools）」は、専門的なAIエージェントを呼び出し可能な関数（ツール）としてラップし、他のエージェントから利用できるようにするAIシステムのアーキテクチャパターンです。これにより、階層的な構造が生まれます：

1. メインの「オーケストレーター」エージェントがユーザーとのやり取りを担当し、どの専門エージェントを呼び出すか判断します

2. 専門的な「ツールエージェント」が、オーケストレーターから呼び出されたときにドメイン固有のタスクを実行します

このアプローチは、人間のチームダイナミクスを模倣しており、マネージャーが専門家を調整し、それぞれが独自の専門知識を活かして複雑な問題を解決します。単一のエージェントがすべてを処理しようとするのではなく、最適な専門エージェントにタスクを委任します。

## 主な利点とコア原則

「ツールとしてのエージェント」パターンには、以下のような利点があります：

- 関心の分離：各エージェントが特定の責任範囲を持つため、システムの理解や保守が容易になります
- 階層的な委任：オーケストレーターがどの専門家を呼び出すか決定し、明確な指揮系統が生まれます
- モジュール型アーキテクチャ：専門家エージェントは、システム全体に影響を与えずに追加・削除・修正が可能です
- パフォーマンス向上：各エージェントは、そのタスクに最適化されたプロンプトやツールを持つことができます

In [ ]:
!pip install -r requirements.txt

In [ ]:
import os

from strands import Agent, tool
from strands_tools import file_write


このモジュールでは、オーケストレーター型のマルチエージェントワークフローを作成します。

<div style="text-align:left">
    <img src="images/architecture.png" width="75%" />

</div>

また、ネストしたエージェントを作成できる `use_llm` についても紹介します。

## リサーチエージェント

まずは http_request ツールを使った基本的なリサーチアシスタントを作成しましょう。

In [ ]:
RESEARCH_ASSISTANT_PROMPT = """You are a specialized research assistant. Focus only on providing
factual, well-sourced information in response to research questions.
Always cite your sources when possible."""

In [ ]:
research_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
    # tools=[http_request]  # Here you can enable an agentic ai search tool
)

query = "Overview of Amazon Bedrock and its features"
# Call the agent and return its response
response = research_agent(query)

このエージェントをツールとしてラップすることで、他のエージェントからも利用できるようになります。

#### ツールとしてのエージェント実装のベストプラクティス

Strandly AIで「ツールとしてのエージェント」パターンを実装する際のポイント：

1. 明確なツールドキュメント：エージェントの専門性を説明する詳細なdocstringを書く
2. フォーカスしたシステムプロンプト：各専門エージェントはドメインに特化させる
3. 一貫したレスポンス処理：レスポンスの抽出・整形パターンを統一する
4. ツール選択の指針：オーケストレーターに、どの専門家をいつ使うか明確な基準を与える

In [ ]:
@tool
def research_assistant(query: str) -> str:
    """
    Process and respond to research-related queries.

    Args:
        query: A research question requiring factual information

    Returns:
        A detailed research answer with citations
    """
    try:
        # Strands agents makes it easy to create a specialized agent
        research_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            system_prompt=RESEARCH_ASSISTANT_PROMPT,
        )

        # Call the agent and return its response
        response = research_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in research assistant: {str(e)}"

次に、ベストプラクティスに従い `product_recommendation_assistant`、`trip_planning_assistant`、`orchestrator` エージェントを作成します。

### 商品レコメンデーションアシスタント

In [ ]:
@tool
def product_recommendation_assistant(query: str) -> str:
    """
    Handle product recommendation queries by suggesting appropriate products.

    Args:
        query: A product inquiry with user preferences

    Returns:
        Personalized product recommendations with reasoning
    """
    try:
        product_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
            system_prompt="""You are a specialized product recommendation assistant.
            Provide personalized product suggestions based on user preferences. Always cite your sources.""",
        )
        # Call the agent and return its response
        response = product_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in product recommendation: {str(e)}"

In [ ]:
product_recommendation_assistant("Product recommendations for flying cars")

### 旅行プランニングアシスタント

In [ ]:
@tool
def trip_planning_assistant(query: str) -> str:
    """
    Create travel itineraries and provide travel advice.

    Args:
        query: A travel planning request with destination and preferences

    Returns:
        A detailed travel itinerary or travel advice
    """
    try:
        travel_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
            system_prompt="""You are a specialized travel planning assistant.
            Create detailed travel itineraries based on user preferences.""",
        )
        # Call the agent and return its response
        response = travel_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in trip planning: {str(e)}"

### オーケストレーターエージェント

In [ ]:
# Define orchestrator system prompt with clear tool selection guidance
MAIN_SYSTEM_PROMPT = """
You are an assistant that routes queries to specialized agents:
- For research questions and factual information → Use the research_assistant tool
- For product recommendations and shopping advice → Use the product_recommendation_assistant tool
- For travel planning and itineraries → Use the trip_planning_assistant tool
- For simple questions not requiring specialized knowledge → Answer directly

Always select the most appropriate tool based on the user's query.
"""

In [ ]:
# Strands Agents allows easy integration of agent tools
orchestrator = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    system_prompt=MAIN_SYSTEM_PROMPT,
    tools=[
        research_assistant,
        product_recommendation_assistant,
        trip_planning_assistant,
        file_write,
    ],
)

In [ ]:
# Example: E-commerce Customer Service System
customer_query = (
    "I'm looking for hiking boots. Write the final response to current directory."
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# The orchestrator automatically determines this requires multiple specialized agents
response = orchestrator(customer_query)

オーケストレーターのメッセージを見てみましょう。ここで、エージェントがサブエージェントをツールとして利用したことが分かります。

In [ ]:
orchestrator.messages

In [ ]:
customer_query = "Can you help me plan my trip to Patagonia"

response = orchestrator(customer_query)

In [ ]:
orchestrator.messages

### 複数エージェントの呼び出し

In [ ]:
orchestrator.messages = []

In [ ]:
query = "Can you do a research on spain? Also help me plan a 7 day trip."

orchestrator(query)

舞台裏では、オーケストレーターが以下の流れで処理します：
1. まず `research_assistant` を呼び出す
2. 次に `trip_planning_assistant` を呼び出す
3. それぞれの専門エージェントの応答を統合し、両方の質問に対応した回答を作成します

### 逐次エージェント連携パターン

エージェントツールは複数のエージェントを組み合わせることも可能です。この例では、`research_agent` の出力を `summary_agent` に渡し、要約された応答を返します。

In [ ]:
 # define the user query
topic = "generative Ai"
# Create a research agent
research_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
)
# Create a summarization agent
summary_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""
    You are a summarization specialist focused on distilling complex information into clear, concise summaries.
    Your primary goal is to take detailed information and extract the key points, main arguments, and critical data.
    You should maintain the accuracy of the original content while making it more digestible.
    Focus on clarity, brevity, and highlighting the most important aspects of the information.
    """,
)

print("Multiple agents created successfully!")
print(f"\n🔍 RESEARCH AGENT working on: {topic}\n") 
try:
    # Agent 1: Invoke research agent
    research_response = research_agent(
        f"Please gather comprehensive information about {topic}."
    )
    research_text = research_response.message.content[0]["text"]
    print("\n✂️ SUMMARY AGENT distilling the research\n")
    
    # Agent 2: Ask the summary agent to create a concise summary
    summary_response = summary_agent(
        f"Please create a concise summary of this research: {research_text}"
    )
    summary_text = summary_response.message.content[0]["text"]
    
    print(summary_text)
except Exception as e:
    print(f"Error in research assistant: {str(e)}")

## おめでとうございます！

Strands Agents でツールとしてエージェントを活用し、より複雑なエージェントアプリケーションを作成する方法を学びました。